# Single facet simulation

In [1]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
import warnings
import matplotlib.pylab as plt

In [2]:
from ibvpy.api import TStepBC
from ibvpy.bcond import BCSlice
from ibvpy.xmodel.xdomain_fe_grid import XDomainFEGrid
from ibvpy.xmodel.xdomain_interface import XDomainFEInterface

In [3]:
from ibvpy.fets import FETS2D4Q
from ibvpy.fets import FETS3D8H
from ibvpy.fets.fets1D5 import FETS1D52ULRH

In [4]:
from bmcs_matmod.slide.vslide_34 import Slide34
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from bmcs_utils.api import Model

In [5]:
from ibvpy.mesh.i_fe_grid_slice import IFENodeSlice
import traits.api as tr

In [6]:
n_e = 1
L_x = 1.0
xd_upper = XDomainFEGrid(coord_min=(0, 0, 0),
                          coord_max=(1, 1, 1),
                          shape=(1, 1, 1),
                          fets=FETS3D8H())
xd_lower = XDomainFEGrid(coord_min=(-1, 0, 0),
                          coord_max=(1, 0, 1),
                          shape=(1, 1, 1),
                          fets=FETS3D8H())

In [7]:
m1 = MATS3DElastic(E=28000, nu=0.3)
m2 = MATS3DElastic(E=28000, nu=0.3)

xd12 = XDomainFEInterface(
    I=xd_lower.mesh.I[:, -1, :],
    J=xd_upper.mesh.I[:, 0, :],
    fets=FETS1D52ULRH()
)

In [16]:
xd12.I.dofs

array([[[30, 31, 32],
        [33, 34, 35]],

       [[42, 43, 44],
        [45, 46, 47]]])

In [10]:
#bond_m = Slide34(E_T=10000, E_N=10000)
material_params = dict(
    E_T=1000, gamma_T=800, K_T=0, S_T=0.0005, c_T=1, bartau=3, 
    E_N=1000, S_N=0.001, c_N = 1, m = 0.01, f_t=3, f_c=20, f_c0=10, eta=0.0 # 0.5
)
bond_m = Slide34(**material_params)

In [11]:
upper_xy = BCSlice(slice=xd_upper.mesh[:, :, :, :, :, :],
                 var='u', dims=[0], value=0.001)
lower_xy = BCSlice(slice=xd_lower.mesh[:, :, :, :, :, :],
                 var='u', dims=[0,1,2], value=0)
bc1 = [upper_xy, lower_xy]

In [12]:
m = TStepBC(
    domains=[
        (xd_upper, m1),
        (xd_lower, m2),
        (xd12, bond_m),
    ],
    bc=bc1,
)
s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.02
s.tstep.fe_domain.serialized_subdomains

In [13]:
m.U_k

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
s.run()

t:	 0.00

LinAlgError: Singular matrix